In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("/content/SMSSpamCollection",sep="\t",names=['Label','Message'])
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
ps=PorterStemmer()
swords=stopwords.words('english')

In [ ]:
def Clean_text(sentence):
  tokens=word_tokenize(sentence)
  clean=[word for word in tokens
         if word.isdigit() or word.isalpha()]
  clean=[word for word in clean
         if word not in swords]
  return clean

In [ ]:
vector=TfidfVectorizer(analyzer=Clean_text)
x=df['Message']
y=df['Label']

In [ ]:
x_new=vector.fit_transform(x)

In [ ]:
x_new

<5572x9764 sparse matrix of type '<class 'numpy.float64'>'
	with 53334 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_new,y,random_state=0,test_size=0.25)

In [ ]:
x_train.shape

(4179, 9764)

In [ ]:
y_train.shape

(4179,)

In [ ]:
x_test.shape

(1393, 9764)

In [ ]:
y.values

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(x_train.toarray(),y_train)

GaussianNB()

In [ ]:
y_pred=nb.predict(x_test.toarray())

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.98      0.92      0.95      1208
        spam       0.63      0.88      0.73       185

    accuracy                           0.91      1393
   macro avg       0.80      0.90      0.84      1393
weighted avg       0.93      0.91      0.92      1393



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1208
        spam       1.00      0.84      0.91       185

    accuracy                           0.98      1393
   macro avg       0.99      0.92      0.95      1393
weighted avg       0.98      0.98      0.98      1393



In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97      1208
        spam       0.99      0.65      0.78       185

    accuracy                           0.95      1393
   macro avg       0.97      0.82      0.88      1393
weighted avg       0.95      0.95      0.95      1393



In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={
    'criterion':['grid','entropy'],
    'max_features':['sqrt','log2'],
    'random_state':[0,1,2,3,4,5],
    'class_weight':['balanced','balanced_subsample']
}

In [ ]:
grid=GridSearchCV(rf,param_grid=parameters,cv=5,scoring='accuracy')

In [ ]:
grid.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ut

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['grid', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'random_state': [0, 1, 2, 3, 4, 5]},
             scoring='accuracy')

In [ ]:
from sklearn.metrics import accuracy_score
rf=grid.best_estimator_
y_pred=rf.predict(x_test)
print("Accuracy Score:",accuracy_score(y_test,y_pred))

Accuracy Score: 0.9798994974874372
